<h1><center>Laboratorio 9: Benchmark Estadístico con Reddit 🧮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Santiago Valdivieso
- Nombre de alumno 2:


### **Link de repositorio de GitHub:** `http://....`

# Temas a tratar

- Optimización de Código en Python.
- Utilización de librerías para medir el tiempo de ejecución de funciones.
- Métodos para optimizar el rendimiento de las funciones.

# Reglas:

- Fecha de entrega: 3/12/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

# Objetivos principales del laboratorio

- Obtener datos desde Reddit a partir de la librería `praw` y visualizar cuales post son más probables que sean puntuados positivamente.
- Aplicar un atajo estadístico para obtener la mean posterior de datos.
- Optimizar a través de librerías funciones.

#Importamos librerias utiles 😸

In [1]:
%%capture
!pip install praw
!pip install line_profiler
!pip install numba

%load_ext autoreload
%autoreload 2
from sklearn.cluster import KMeans
import sys
import praw
import time
import numpy as np
import pandas as pd
from numba import jit
import plotly.express as px
from functools import lru_cache
from IPython.core.display import Image

# 1. Recomendando Posts de Subreddit

<p align="center">
  <img src="https://styles.redditmedia.com/t5_2rer8/styles/bannerBackgroundImage_xzrb6my122181.jpg">
</p>

Unos intrépidos alumnos del curso, quienes frecuentemente participan en subreddits y foros de reviews, se preguntan lo siguiente: ¿Podremos confiar que un post es bueno, si este tiene solamente 1 o 3 votos positivos?. los compañeros, creen que esto claramente no representa una opinión general, ya que estamos mucho menos seguros acerca de la verdadera proporción de votos a favor de los comentarios con pocos datos. ¿Pero cómo podemos obtener una representación más creíble para este problema?.

Lo señalado forma parte de un problema estadísticos, donde a través del cálculo de la posterior se puede conocer que tan probable es que un post sea bueno. Para efectos de este laboratorio, no se exige un conocimiento previo para resolver este problema, simplemente se deberá aplicar las ecuaciones presentadas más adelante (De igual forma si quedan interesados sobre el tema se les invita a tomar el ramo [CC6104](https://github.com/dccuchile/CC6104)).

## 1.1 Obtención de Subrredits y Análisis [1 Punto]

Para estudiar que tan probable es que un post sea positivo se comenzará cargando datos reales del subreddit chile desde reddit (si usted desea puede cambiar el subreddit a uno de su gusto). Para esto le proponemos la utilización de la función que aparece mas abajo, la que presenta un usuario ya creado por el equipo docente. Dese un tiempo para entender que hace cada parte de la función, visualizando que se obtiene de estas.

Revisada la función, utilice un **perfilador** para monitorear el tiempo y memoria que les toma a cada liena de código para ser ejecutada. Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.

**TO-DO:**
- [ ] Estudiar la función propuesta por el equipo docente.
- [ ] Estudiar los tiempos de ejecución del código a través de un perfilador.
- [ ] Estudiar la memoria ocupada por el código a través de un perfilador.
- [ ] Comente los resultados. ¿Qué lineas ocupan más memoria o más/menos tiempo?¿Por qué?

In [2]:
%load_ext line_profiler

In [3]:
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url


In [4]:
votes, post, url = praw_reddit()

In [5]:
%lprun -f praw_reddit praw_reddit()

In [6]:
%%file bench_module.py
import numpy as np
import praw
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

Overwriting bench_module.py


In [25]:
%load_ext memory_profiler

In [26]:
from bench_module import praw_reddit
%mprun -f praw_reddit praw_reddit()


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 845, in enable
    sys.settrace(self.trace_memory_usage)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 848, in disable
    sys.settrace(self._original_trace_function)



**Respuesta:**

La línea que ocupa más tiempo de procesamiento de la función praw_reddit es la 13 "top_submissions = list(subreddit.hot(limit=n_hot))" la cual tarda poco más de 13 segundos, esto ocurre porque debe extraer 1000 submisions de reddit conectándose a la red. Esta es la que toma más tiempo del código 99,3%.

Por otro lado, las líneas que tardan menos tiemmpo son de la 3 a la 7 y la 33 que es el return, que demoran 1 micro segundo o menos, que son las variables con las cuales se entra a reddit, por lo que representan sólo una definición de una variable para el procesamiento y el return que simplemente devuelve el valor que ya se tiene guardado, por lo que es exigente en rendimiento.

Respecto a la memoria la línea que más ocupa es la primera de la función, "def praw_reddit(nombre_subreddit="chile", n_hot=1000):" la cual toma 406,2 Mib, en usarla, sin embargo esta no aumenta en las siguientes líneas porque los numeros  de ocurrencia son valores muy bajos(ninguno mayor a 10000)



## 1.2 LRU y Análisis de Tiempo con Cache [1 Punto]

Respondan las siguientes preguntas:

1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.

Respondidas las preguntas, se le solicita que aplique alguna técnica de caching para mejorar el desempeño de la función `praw_reddit`. Para esto compare solo el tiempo de ejecución del algoritmo con y sin caching, señalando el tiempo total de ejecución y el tiempo promedio que le toma ejecutar cada loop a la función. Con lo anterior, ¿es posible visualizar mejoras en este caso?.


**TO-DO:**
- [ ] Responder las preguntas.
- [ ] Mejorar el código con cache.
- [ ] Explicar las consecuencias de usar lru_cache sobre praw_reddit

**Respuestas Teóricas:**

`Escriba su respuesta aquí`


¿Cuál es la consecuencía de ocupar caching en la función anterior?.

la memoria cache es un espacio que puede estar ubicado en memoria (RAM), disco o almacenada de manera remota. El acto de guardar resultados en memoria para luego utilizarlos de manera directa se denomina memorización y es una forma de chaching para mejorar el rendimiento de aplicaciones por medio de un uso eficiente de la memoria, una de las ideas tras esta premisa es la de guardar los resultados de operaciones intensivas. Las siglas LRU representan e l algoritmo Least Recently Used, el cual descarta primero los elementos menos usados recientemente. El algoritmo lleva el seguimiento de lo que se va usando, lo que resulta caro si se quiere hacer con precisión. La implementación de esta técnica requiere llevar la cuenta de la edad de cada elemento de caché y buscar el menos usado sobre la base de ella.

Las técnicas de caching tienen un costo, este radica en aumentar el consumo de memoria, si esta memoria esta localizada en disco, el acceso puede ser muy lento y el rendimiento puede decaer drásticamente.

In [8]:
from functools import lru_cache

In [9]:
@lru_cache(maxsize=1000)
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url


In [10]:
%%timeit
votes, post, url = praw_reddit()

The slowest run took 36986631.62 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 356 ns per loop


**Respuesta de lru_cache sobre praw_reddit**
Podemos ver que al aplicar %%timeit que el tiempo que demoró la iteración más lenta de la función es 63309883.73 veces mayor que la más rápida. esto es porque el resultado de la función está guardada en el cache. En este caso en particular debido a que se van generando datos por distintos tamaños de batches, se repiten las funciones distintas veces, lo que hace que al guardarlas en el caché el rendimiento total sea mucho mayor

## 1.3 Obtención de Mean Posterior y Standard Error [1 Punto]

Una forma de obtener la mean posterior y el Standard Error de los datos de reddit es aplicando un atajo de inferencia estadística. Este atajo se define de la siguiente forma:

Sea **u** los votos positivos y **d** los votos negativos del subreddit, tendremos que:

$$a = 1+u$$

$$b = 1+d$$

$$\sigma= 1.65\sqrt(\dfrac{ab}{(a + b)^2(a + b + 1)})$$

$$\mu = \dfrac{a}{a+b}$$

Donde $\mu$ es la mean posterior y $\sigma$ el standard error.

Con lo anterior, genere dos funciones que tengan como salida $\mu$ y $\sigma$ de acuerdo a las ecuaciones señaladas. La primera función, deberá ser construida sin el uso de numpy, aplicando for y aplicando comandos nativos de Python. Por otro lado, deberá generar una segunda función con el uso exclusivo de numpy. **OJO** que las funciones deben tener como entrada solo un elemento del diccionario votes (por ejemplo `votes[1]`), por lo que estas no deben tener como entrada el conjunto completo de datos.

In [11]:
def funcion11(votos):
  Media=[]
  desv=[]
  for i in range(len(votos)):
    positivos = votos[i][0]
    negativos = votos[i][1]
    a=1+positivos
    b=1+negativos
    media=a/(a+b)
    desviación=1.65*(((a*b)/(((a*b)**2)*(a+b+1)))*0.5)
    Media.append(media)
    desv.append(desviación)

  return Media,desv

In [12]:
def funcion22(votos):
  a=1+votos[:,0]
  b=1+votos[:,1]
  media=a/(a+b)
  desviación=1.65*(((a*b)/(((a*b)**2)*(a+b+1)))*0.5)
  return media, desviación

In [13]:
%timeit funcion11(votes[0])

1000 loops, best of 5: 161 µs per loop


In [14]:
%timeit funcion22(votes[0])

The slowest run took 7.65 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 13.2 µs per loop


In [15]:
funcion11(votes[0])

([0.822429906542056,
  0.9389067524115756,
  0.9186046511627907,
  0.9813664596273292,
  0.8388888888888889,
  0.8769230769230769,
  0.9345794392523364,
  0.9259259259259259,
  0.9741379310344828,
  0.9467213114754098,
  0.9142857142857143,
  0.9494949494949495,
  0.8961038961038961,
  0.7924528301886793,
  0.8918918918918919,
  0.8333333333333334,
  0.8695652173913043,
  0.8787878787878788,
  0.6770833333333334,
  0.7843137254901961,
  0.9385593220338984,
  0.9565217391304348,
  0.9166666666666666,
  0.875,
  0.8105263157894737,
  0.9264705882352942,
  0.9789473684210527,
  0.9,
  0.9758620689655172,
  0.6666666666666666,
  0.8762886597938144,
  0.8333333333333334,
  0.917910447761194,
  0.9532163742690059,
  0.9794584500466853,
  0.918918918918919,
  0.8913043478260869,
  0.8823529411764706,
  0.9130434782608695,
  0.8669527896995708,
  0.972027972027972,
  0.8701298701298701,
  0.7368421052631579,
  0.9259259259259259,
  0.8,
  0.9259259259259259,
  0.9953917050691244,
  0.777777777

In [16]:
funcion22(votes[0])

(array([0.82242991, 0.93890675, 0.91860465, 0.98136646, 0.83888889,
        0.87692308, 0.93457944, 0.92592593, 0.97413793, 0.94672131,
        0.91428571, 0.94949495, 0.8961039 , 0.79245283, 0.89189189,
        0.83333333, 0.86956522, 0.87878788, 0.67708333, 0.78431373,
        0.93855932, 0.95652174, 0.91666667, 0.875     , 0.81052632,
        0.92647059, 0.97894737, 0.9       , 0.97586207, 0.66666667,
        0.87628866, 0.83333333, 0.91791045, 0.95321637, 0.97945845,
        0.91891892, 0.89130435, 0.88235294, 0.91304348, 0.86695279,
        0.97202797, 0.87012987, 0.73684211, 0.92592593, 0.8       ,
        0.92592593, 0.99539171, 0.77777778, 0.81052632, 0.625     ]),
 array([4.56871345e-06, 5.96718439e-08, 1.38455923e-07, 1.07438662e-05,
        1.30470345e-07, 3.45269184e-06, 1.09126984e-05, 9.46100917e-06,
        2.08002420e-05, 1.12132765e-06, 9.00812369e-06, 1.75531915e-05,
        1.91610925e-05, 3.30687831e-05, 1.64473684e-04, 1.23586248e-05,
        7.31382979e-05, 2.0917

## 1.4 Comparación de Rendimiento [2 Puntos]

Creadas las funciones de la sección 1.3, de forma iterativa compare los diferentes batches de datos que contiene el diccionario `votes`. Para esto genere un gráfico utilizando plotly, donde se pueda observar las diferencias de tiempo que toma ejecutar las diferentes cantidades de datos. ¿es posible observar una diferencia? ¿a qué se deberá esto?.

Aplique el compilador **Numba** sobre las funciones 1.3 y compare el tiempo de ejecución con los obtenidos sin el compilador. 

In [17]:
from numba import jit
@jit(nopython=True)
def funcion11_numba(votos):
  positivos=0
  negativos=0
  for i in range(len(votos)):
    positivos = positivos+ votos[i][0]
    negativos = negativos + votos[i][1]
  a=1+positivos
  b=1+negativos
  media=a/(a+b)
  desviación=1.65*(((a*b)/(((a*b)**2)*(a+b+1)))*0.5)
  return media, desviación

@jit(nopython=True)
def funcion22_numba(votos):
  a=1+np.sum((votos)[:,0])
  b=1+np.sum((votos)[:,1])
  media=a/(a+b)
  desviación=1.65*(((a*b)/(((a*b)**2)*(a+b+1)))*0.5)
  return media, desviación

In [21]:
tiempo2=[]
for i in range(len(votes)):
  t0 = time.process_time()
  funcion11(votes[i])
  t1 = time.process_time()
  funcion11_numba(votes[i])
  t2 = time.process_time()
  funcion22(votes[i])
  t3 = time.process_time()
  a=np.array(votes[i])
  t4 = time.process_time()  
  funcion22_numba(a)
  t5 = time.process_time()
  tiempo2.append([len(votes[i]),(t1-t0),(t2-t1),(t3-t2),(t5-t4)])

df = pd.DataFrame(np.array(tiempo2), 
             columns=['Cantidades', 
                      'Función Nativa',
                      'Función Nativa Numba', 
                      'Función Numpy',
                      'Función Numpy Numba'])

In [22]:
fig = px.line(df, x="Cantidades", y=['Función Nativa','Función Nativa Numba','Función Numpy','Función Numpy Numba'])
fig.show()

Ahora, grafiquen los tiempos tomados y comente los desempeños.

**Nota:** Ejecuten las funciones compiladas con algún ejemplo antes de graficar. De lo contrario, les indicará el tiempo de compilación, cosa que no queremos medir

Podemos ver que el orden de rendimiento de las funciones son las Numba tanto en numpy como nativa son las más rápidas. Luego la función cn nupy y finalmente la que más aumenta el tiempo de ejecución según la cantidad de elementos es la nativa.

Si se hac zoom en el gráfico (no se alcanza a apreciar en sin zoom) la función numpy usando numba es más rápida que la nativa.

## 1.5 Plot de Resultados [1 Punto]

Llego la hora de visualizar los resultados obtenidos, para esto solo ejecute las siguientes celdas y observe lo que sucede :3. ¿Qué logra observar de los resultados?, ¿La solución resulta trivial?

En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

In [23]:
votos = votes[len(votes) - 1]
posteo = post[len(post) - 1]
print("lower bounds aproximados:")
posterior_mean, std_err = funcion22(votos)
lb = posterior_mean - std_err
print("\nTop 20 post ordenador por el limite inferior:\n")
order = np.argsort(-lb)
vote_post = {
    'Votos (+)': votos[order[:20], 0],
    'Votos (-)': votos[order[:20], 1],
    'Post': np.array(posteo)[order[:20]],
    'url': np.array(url[len(url) - 1])[order[:20]]
}
df = pd.DataFrame(data=vote_post)
ordered_post = df.Post

r_order = order[::-1][-20:]
data_dic = {
    'mean': posterior_mean[r_order],
    'std_err': std_err[r_order],
    'post': ordered_post[::-1]
}
df = pd.DataFrame(data=data_dic)
fig = px.scatter(df, x="mean", y="post", error_x="std_err")
fig.show()

lower bounds aproximados:

Top 20 post ordenador por el limite inferior:



**Respuesta:**

En la visualización se puede apreciar los 20 post de reddit ordenados de mayor a menor por su media posterior, la que estadísticamente representa la probabilidad un post sea bueno. Podemos ver que el post con mayor probalididad es que le guste al lector es el que empieza con ¡Estamos de vuelrta!, con un valor de 0.998% de probabilidad de que al lector de lo considere positivo. En cambio el post número 20 dentro de este ranking es "El crossover que nadie pidió, pero que todos necesitabamos". 

Además podemo obtener las desviaciones estandar de cada uno de estos post, la con mayor desvuación es la de "Sernac acusa a Banco de Chile (...)". 

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>